## Initialisations

In [ ]:
TAP_URL = "http://archive.eso.org/tap_obs"
TAP_URL_SIMBAD = 'https://simbad.unistra.fr/simbad/sim-tap'

# Importing useful packages
import os 
import sys
import requests
#import cgi
import json
import time
from pathlib import Path
from astropy.table import Table, vstack, unique

import pyvo
from pyvo.dal import tap
from pyvo.auth.authsession import AuthSession
    
# Verify the version of pyvo 
from pkg_resources import parse_version
pyvo_version = parse_version(pyvo.__version__) 
test_pyvo_version = (pyvo_version == parse_version('1.1') or pyvo_version > parse_version('1.2.1') )
if not test_pyvo_version:
    print('You are using a not supported version of pyvo (version={version}).\nPlease use pyvo v1.1, v1.3, or higher, not v1.2* [ref. pyvo github issue #298]'.format(version=pyvo.__version__))
    raise ImportError('The pyvo version you are using is not supported, use 1.3+ or 1.1.')

import eso_programmatic as eso

In [ ]:
def run_tap_query(tap, query):
    results = None

    # define a job that will run the query asynchronously 
    job = tap.submit_job(query)
    
    # extending the maximum duration of the job to 300s (default 60 seconds)
    job.execution_duration = 300 # max allowed: 3600s
    
    # job initially is in phase PENDING; you need to run it and wait for completion: 
    job.run()
    
    try:
        job.wait(phases=["COMPLETED", "ERROR", "ABORTED"], timeout=600.)
    except pyvo.DALServiceError:
        print('Exception on JOB {id}: {status}'.format(id=job.job_id, status=job.phase))
    
    print("Job: %s %s" %(job.job_id, job.phase))
    
    if job.phase == 'COMPLETED':
        # When the job has completed, the results can be fetched:
        results = job.fetch_result()
    
    # the job can be deleted (always a good practice to release the disk space on the ESO servers)
    job.delete()

    return results

## Prompt for user's credentials and get a token

In [ ]:
import getpass

username = input("Type your ESO username (blank for unauthenticated): ")
if username:
    password=getpass.getpass(prompt="%s's password: "%(username), stream=None)

    token = eso.getToken(username, password)
    if token != None:
        print('token: ' + token)
        session = requests.Session()
        session.headers['Authorization'] = "Bearer " + token

        tap = pyvo.dal.TAPService(TAP_URL, session=session)
    else:
        sys.exit(-1)
else:
    print('OK, no proprietary data then ...')
    token = ''
    tap = pyvo.dal.TAPService(TAP_URL)

## Resolve the target name with Simbad

In [ ]:
tap_simbad = pyvo.dal.TAPService(TAP_URL_SIMBAD)

target = 'SN1987A'

query_simbad = """
SELECT ra, dec, main_id
  FROM basic
  WHERE main_id = '""" + target + """'
"""

simbad = run_tap_query(tap_simbad, query_simbad)
ra = str(simbad['ra'].data[0])
dec = str(simbad['dec'].data[0])

## Run the query to the ESO archive

In [ ]:
# define the query you want to run, e.g.:

query = """
SELECT TOP 4 *
FROM dbo.raw
WHERE dp_cat = 'SCIENCE'
AND instrument = 'UVES'
AND  INTERSECTS(s_region, CIRCLE('ICRS', """ + ra + """, """ + dec + """, 100./3600.)) = 1
                           -- a circle centered onto SN1987A, of radius=100 arcsec,
AND date_obs > '2015'
"""

In [ ]:
results = tap_query(tap, query)

# Let's print the results to examine the content:
# check out the access_url and the datalink_url
if results:
    print('%i matching files found' % len(results))
    # eso.printTableTransposedByTheRecord(results.to_table()) 
else:
    print("!" * 42)
    print("!                                        !")
    print("!       No results could be found.       !")
    print("!       ? Perhaps no permissions ?       !")
    print("!       Aborting here.                   !")
    print("!                                        !")
    print("!" * 42)
    quit()

## Download the selected science files ...

In [ ]:
dirname = input("Directory to save the files in: ")
Path(dirname).mkdir(parents=True, exist_ok=True)

print("Start downloading...")
for raw in results:
    access_url = raw['access_url'] # the access_url is the link to the raw file
    if username:
        status, filepath = eso.downloadURL(access_url, session=session, dirname=dirname)
    else:
        status, filepath = eso.downloadURL(access_url, dirname=dirname)
    if status==200:
        print("      RAW: %s downloaded  "  % (filepath))
    else:
        print("ERROR RAW: %s NOT DOWNLOADED (http status:%d)"  % (filepath, status))
print('... download complete')

## ... and the corresponding calibrations

In [ ]:
mode_requested = 'raw2raw'
semantics = 'http://archive.eso.org/rdf/datalink/eso#calSelector_' + mode_requested

uniqe_calib_urls = Table(names=('access_url', 'eso_category'), dtype=('object', 'object'))

for result in results:
    print('Processing raw science file %s for calibrations' % result['dp_id'])
    datalink_url = result['datalink_url']

    if username:
        datalink = pyvo.dal.adhoc.DatalinkResults.from_result_url(datalink_url, session=session)
    else:
        datalink = pyvo.dal.adhoc.DatalinkResults.from_result_url(datalink_url)

    calsel_url = next(datalink.bysemantics(semantics)).access_url

    if username:
        associated_calib_files = pyvo.dal.adhoc.DatalinkResults.from_result_url(calsel_url, session=session)
    else:
        associated_calib_files = pyvo.dal.adhoc.DatalinkResults.from_result_url(calsel_url)

    # create and use a mask to get only the #calibration entries,
    # given that other entries, like #this or ...#sibiling_raw, could be present:
    calibrator_mask = associated_calib_files['semantics'] == '#calibration'
    calib_urls = associated_calib_files.to_table()[calibrator_mask]['access_url','eso_category']

    uniqe_calib_urls = unique(vstack([uniqe_calib_urls, calib_urls]), keys='access_url')

i_calib=0
for url, category in uniqe_calib_urls:
    i_calib+=1
    status, filename = eso.downloadURL(url, dirname=dirname)
    if status==200:
        print("       ... %4d/%d dp_id: %s (%s) downloaded"  % (i_calib, len(uniqe_calib_urls), filename, category))
    else:
        print("       ... %4d/%d dp_id: %s (%s) NOT DOWNLOADED (http status:%d)"  % (i_calib,len(unique_calib_urls), filename, category, status))

print('... download done!')


## All done!